In [1]:

# import json
# import pickle
# import logging
import os, sys
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import transformers

import torch
from torch import nn
import torch.nn.functional as F
# import torch.optim as optim
from torch.utils.data import Dataset #, DataLoader
import gluonnlp as nlp
import numpy as np
from collections import defaultdict

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import BertTokenizer, BertModel

from pathlib import Path
from utils import BERTClassifier

import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda:0")
    

C:\Users\KIMJOONHONG\Anaconda3\envs\JH\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
def main():
    # _model_dir = "/home/k4ke/kobert/saves"
    # model_dir = Path(_model_dir)
    # model_config = Config(json_path = model_dir / 'config.json')

    # Vocab and Tokenizer
    tokenizer = get_tokenizer()
    
    bertmodel, vocab = get_pytorch_kobert_model()
    # token_to_idx = vocab.token_to_idx
    #
    # # vocab_size = len(token_to_idx)
    # print("len(toekn_to_idx): ", len(token_to_idx))
    #
    # with open(model_dir / "token2idx_vocab.json", 'w', encoding='utf-8') as f:
    #     json.dump(token_to_idx, f, ensure_ascii=False, indent=4)
    #
    # # save vocab & tokenizer
    # with open(model_dir / "vocab.pkl", 'wb') as f:
    #     pickle.dump(vocab, f)
    #
    # # load vocab & tokenizer
    # with open(model_dir / "vocab.pkl", 'rb') as f:
    #     vocab = pickle.load(f)

    # tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=64)
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    model = BERTClassifier(bertmodel)

    # load model
    model_dict = model.state_dict()
    # checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))
    # checkpoint = torch.load("/home/k4ke/kobert/saves/best-epoch-5-f1-0.916.bin", map_location = torch.device('cpu'))
    checkpoint = torch.load("C:/Users\KIMJOONHONG/ml/datasets/saves/best-epoch-13-f1-0.715.bin", map_location=torch.device('cpu'))
    convert_keys = {}
    for k, v in checkpoint['model_state_dict'].items():
        new_key_name = k.replace("module.", '')
        if new_key_name not in model_dict:
            print("{} is not int model_dict".format(new_key_name))
            continue
        convert_keys[new_key_name] = v

    model.load_state_dict(convert_keys)
    # model.load_state_dict(checkpoint)

    model.eval()
    model.to(device)
    #model.resize_token_embeddings(len(tokenizer))
    emo_dict = {0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}


    while(True):
        _sentence = input("input: ")
        if _sentence == '-1':
            break
        transform = nlp.data.BERTSentenceTransform(tok, max_seq_length=64, pad=True, pair=False)
        # self.sentences = [transform([i[sent_idx]]) for i in dataset]
        sentence = [transform([_sentence])]
        #data_train = BERTDataset(sentence, 0, 1, tok, 64, True, False)
        dataloader = torch.utils.data.DataLoader(sentence, batch_size=1)
        _token_ids = dataloader._index_sampler.sampler.data_source
        # print(_token_ids)
        # print(_token_ids[0])
        # print(_token_ids[0][0])
        _t = torch.from_numpy(_token_ids[0][0])
        _t = _t.tolist()
        token_ids = torch.tensor(_t, dtype=torch.long).unsqueeze(0).cuda()
        val_len = torch.tensor([len(token_ids[0])], dtype=torch.long).cuda()
        # val_len = torch.tensor([len(token_ids)], dtype=torch.long).cuda()

        _s = torch.from_numpy(_token_ids[0][1])
        _s = _s.tolist()
        segment_ids = torch.tensor(_s, dtype=torch.long).unsqueeze(0).cuda()
        # segment_ids = torch.from_numpy(_token_ids[0][1]).unsqueeze(0)
        # segment_ids = torch.zeros()
        # print(len(token_ids)) # 1

        out = model(token_ids, val_len, segment_ids)
        out_idx = np.argmax(out.cpu().detach().numpy())
        softmax = nn.Softmax(dim=1)
        score = softmax(out).cpu().detach().numpy()

        print("out: ", out)
        print(out_idx, emo_dict[out_idx])
        print("score: ", score)

if __name__ == "__main__":
    main()
    print("done")

using cached model
using cached model
using cached model
input: 나 지금 너무 눈물나 ㅠㅠ
out:  tensor([[ 1.2232, -0.0669, -1.8283,  4.9234,  1.5774, -2.1032, -2.7579]],
       device='cuda:0', grad_fn=<AddmmBackward>)
3 슬픔
score:  [[2.3118014e-02 6.3633285e-03 1.0931710e-03 9.3521851e-01 3.2945052e-02
  8.3044672e-04 4.3149601e-04]]
input: 아 뭐 어쩌라고 나한테!!
out:  tensor([[-2.3251,  3.0880,  3.4174, -2.0633,  2.1222, -2.1368, -2.2351]],
       device='cuda:0', grad_fn=<AddmmBackward>)
2 분노
score:  [[0.001597   0.35824665 0.49802276 0.0020749  0.13638349 0.00192793
  0.00174735]]
input: 자기야 사랑해 ><
out:  tensor([[-1.1516,  6.9373, -1.9279, -1.8856,  1.8791, -2.5480, -2.0795]],
       device='cuda:0', grad_fn=<AddmmBackward>)
1 놀람
score:  [[3.04731919e-04 9.92901206e-01 1.40217657e-04 1.46273203e-04
  6.31167274e-03 7.54214780e-05 1.20486395e-04]]
input: 특이하네 이건 소금이 안들어가네
out:  tensor([[-1.2446,  7.0159, -1.8439, -1.8639,  1.6631, -2.4347, -2.0818]],
       device='cuda:0', grad_fn=<AddmmBackward>)
1 놀